## CSE202 - Disign and Analysis of Algorithms - Week 8 - Amortization 摊销

---

日期：2022-11-11

姓名：Yubo Cai

课程：CSE202 - Disign and Analysis of Algorithms 高级算法设计与分析

<img src="https://upload.wikimedia.org/wikipedia/commons/0/06/%C3%89cole_polytechnique_signature.svg" style="float: left; width: 15%" />

In [2]:
from sympy.matrices import randMatrix
import numpy as np
import matplotlib.pyplot as plt
import timeit
import math
import sympy as sym
import crypt

`Amortized`: average the worst case over a sequence of operations

`摊销`: 对一系列操作的最坏情况取平均

<img src="./img/pic1.png" style="float: left; width: 20%" />

### 1. Dynamic Tables

<img src="./img/pic2.png" style="float: left; width: 45%" />

Increasing the size of the table requires: allocating a new array of memory; `copying` the old array to the new one.

In [1]:
def __init__(self):
    self.size = 0
    self.capacity = 0
    self.table = []


def __getitem__(self, i):
    if i >= self.size:
        raise IndexError
    return self.table[i]


def __setitem__(self, i, v):
    if i >= self.size:
        raise IndexError
    self.table[i] = v


def append(self, v):
    n = self.size
    self.resize(n + 1)
    self.table[n] = v


def resize(self, newsize):
    if newsize > self.capacity:
        self.realloc((int)(alpha * newsize)) # In python the alpha is 9/8   
    self.size = newsize

#### 1.1 Amortized Cost of a Sequence of Append Operations

Squence of capacities:
$$
t_{k+1}=\left\lfloor\alpha\left(t_k+1\right)\right\rfloor, \quad with\ \alpha>1
$$
There, $\left\lfloor\alpha\left(t_k+1\right)\right\rfloor$ is a direct translation of the statement (int) (alpha*newsize) and the fact that newsize, the argument of resize, takes for values successive integers, so that the first time it is larger than the capacity $t_k$ is when it equals $t_k+1$.
Thus, we need to compute an upper bound for
$$
S_N:=\sum_{t_k \leq N} t_k .
$$
We denote by $m$ the index of the last $t_k$ in the sum, for which we have $t_m \leq N$ (we have that $\left.N<t_{m+1}\right)$. Using the inequality $x-1<\lfloor x\rfloor$, the recurrence gives
$$
\alpha\left(t_{k-1}+1\right)-1<t_k
$$
whence, since $\alpha>1$,
$$
t_{k-1}<\frac{1}{\alpha}\left(t_k+1\right)-1<\frac{1}{\alpha} t_k
$$
By induction, we can show that
$$
t_k<\frac{t_m}{\alpha^{m-k}}
$$
which, because $t_m \leq N$ and $\alpha>1$, implies that
$$
t_k \leq \frac{N}{\alpha^{m-k}}
$$
Finally,
$$
S_n=t_m+t_{m-1}+t_{m-2}+\cdots \leq N\left(1+\frac{1}{\alpha}+\frac{1}{\alpha}^2+\cdots\right)=N \frac{\alpha}{\alpha-1} .
$$
In the last step, we use the fact that the geometric series converges to $1 /(1-1 / \alpha)$ since $1 / \alpha<1$

`Theorem`: The amortized cost of a sequence of $N$ append operations is bounded by 
$$
\frac{C_{N}}{N} \leq 1+\frac{\alpha}{\alpha-1} 
$$

<img src="./img/pic3.png" style="float: left; width: 20%" />

#### 1.2 Deletion

Retrieve memory when the size of the table decreases

`Solution`: leave space to prepay for the next growth.

We consider a sequence of $N$ operations of type append or delete, with the algorithm that changes the capacity from $N$ to $\alpha N$ when the table is full and from $N$ to $\alpha \beta N$ when it has at most $\beta N$ elements (and necessarily $\alpha>1$ to enable growth and $\alpha \beta<1$ to allow shrinking).

First, observe that after a resizing of the table, following an insertion or a deletion, its filling ratio is always $1 / \alpha$. If $N$ denotes the current capacity, the cost that should be charged to each append operation is as before 1 for the element itself, 1 for its future copy and a cost for the future copy of the $N / \alpha$ elements that are already there in the table, divided by the minimal number of elements that will be appended before the next copy due to an append, which is $N(1-1 / \alpha)$. This gives a charge
$$
1+1+\frac{N / \alpha}{N(1-1 / \alpha)}=1+\frac{\alpha}{1-\alpha}
$$
Similary, the cost that should be charged to each delete operation is 1 for its deletion itself, plus a cost for the future copy of the $\beta N$ elements, divided by the minimal number of elements that will be deleted before that next copy due to a delete, which is $(1 / \alpha-\beta) N$. This gives a charge
$$
1+\frac{\beta N}{(1 / \alpha-\beta) N}=1+\frac{\alpha \beta}{1-\alpha \beta} .
$$
Thus we have established that the amortized cost of a sequence of $N$ append or delete is $O(N)$.

### 2. Union-Find 联合查找， 并查集

Abstract data type for `Equivalence Relations` 等价关系

A binary relationship ~ between two objects is said to be an equivalence relation if :

- Reflexive: x~x
- Symmetric: x~y $\Rightarrow$ y~x
- Transitive: x~y $\wedge$ y~z $\Rightarrow$ x~z

Find(p): return the identifier of the set containing p

Union(p, q): merge the sets containing p and q

#### 2.1 Forests of Trees

<img src="./img/pic4.png" style="float: left; width: 40%" />

#### 2.2 Union by Rank

Maintain `rank` (=height). Link short trees to higher ones.

<img src="./img/pic5.png" style="float: left; width: 40%" />

$A_{a}$, A是指group, a是这个group的rank. 在图中我们有0~1, 2~3, 0~3这三个等价类，因此能够根据这三个等价类形成三个tree.

`Properties.`
- **The size of a subtree is at least $2^{rank(root)}$**. Again, by induction on the steps of the algorithms. Initially, all trees have size 1 and a root of rank 0; the property holds. Consider now two trees whose roots have rank $r_1$ and $r_2$. By the induction hypothesis, they have subtrees with at least $2^{r_1}$ and $2^{r_2}$ nodes. If $r_1 \neq r_2$, then the link operation does not change any of the ranks; the size of the subtree of the root with the smaller rank is unchanged and the other one is increased, so that the property is preserved. If $r_1=r_2$ then one of the nodes becomes the child of the other one and keeps its rank and subtree. The other one gets rank $r_1+1$ and its subtree has size at least $2^{r_1}+2^{r_2}=2^{r_1+1}$, which concludes the proof.

- **The number of nodes or rank r is $\leq \frac{n}{2^r}$** By the previous statement, each node of rank $r$ has a subtree of size at least $2r$. Thus the total number of nodes of that rank is at most $\frac{n}{2^r}$

- **The number of nodex of rank > s is $\leq n/2^{s}$**. This is a consequence of the previous one that is used on slide 18. It is obtained by summing the previous bound for $r = s + 1, s + 2, . . . .$

`The worst case for find:` $$O(\log n)$$

#### 2.3 Path Compression 路径压缩

<img src="./img/pic7.png" style="float: left; width: 40%" />

Every find branches all the nodes it visits to their root.

As the graph shown above that we want to union 3 and 6. We have the root of 3 is 0 and the root of 6 is 5. Therefore, as the path compression. we make $3 \rightarrow 0$ and $6 \rightarrow 5$. Then we merge the root 0 and 5 together where we get the graph like following.

#### 2.4 Link \& Compress

1. Rewrite the sequence of $m$ union or find as a sequence of $O(m)$ `link` or `Compress`.
$$
\begin{gathered}
l(0,1), l(2,3), l(0,2), l(5,4), \\
l(7,6), l(5,7), \underbrace{c(3,0), c(6,5), l(0,5)}_{\text {union }(3,6)}
\end{gathered}
$$
2. Perform the link first (each in $O(1)$ operations)

<img src="./img/pic6.png" style="float: left; width: 50%" />


**Definition.** `T(m, n, r)` worst-case number of parent changes in $\leq m$ compress in a forest of $\leq n$ nodes, each of rank $\leq r$.

With Simple bound  that
$$
T(m,n,r) \leq nr.
$$
<img src="./img/pic8.png" style="float: left; width: 25%" />

Therefore as the graph shown above, we have the worst case of $T(m,n,r) \leq nr$ where $n$ is the number of nodes, $r$ is the rank of the node and $m$ is the number of compress. In the worst case that we have to make all the nodes to the root, which give us the bound.